In [1]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use basic agent operations from
    the Azure Agents service using a synchronous client.

USAGE:
    python sample_agents_basics.py

    Before running the sample:

    pip install azure-ai-projects azure-identity

    Set this environment variables with your own values:
    PROJECT_CONNECTION_STRING - the Azure AI Project connection string, as found in your AI Foundry project.
"""

'\nDESCRIPTION:\n    This sample demonstrates how to use basic agent operations from\n    the Azure Agents service using a synchronous client.\n\nUSAGE:\n    python sample_agents_basics.py\n\n    Before running the sample:\n\n    pip install azure-ai-projects azure-identity\n\n    Set this environment variables with your own values:\n    PROJECT_CONNECTION_STRING - the Azure AI Project connection string, as found in your AI Foundry project.\n'

In [2]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageTextContent
import requests
import sys
import os
import logging
sys.path.append("..")
from utils import load_azd_env, set_azure_ai_project_connection_string, setup_logging, get_file_paths

In [3]:
load_azd_env()
set_azure_ai_project_connection_string()


In [4]:
enbale_logging = False

if enbale_logging:
    setup_logging()

In [5]:
# [START create_project_client]
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AZURE_AI_FOUNDRY_CONNECTION_STRING"],
)
# [END create_project_client]

In [6]:
with project_client:

    # [START create_agent]
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_AI_DEPLOYMENT_MODEL"],
        name="my-assistant",
        instructions="You are helpful assistant",
    )
    # [END create_agent]
    print(f"Created agent, agent ID: {agent.id}")

    # [START create_thread]
    thread = project_client.agents.create_thread()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

    # [START create_message]
    message = project_client.agents.create_message(thread_id=thread.id, role="user", content="Hello, tell me a joke")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

    # [START create_run]
    run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)

    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # [START list_messages]
    messages = project_client.agents.list_messages(thread_id=thread.id)

    # The messages are following in the reverse order,
    # we will iterate them and output only text contents.
    for data_point in reversed(messages.data):
        last_message_content = data_point.content[-1]
        if isinstance(last_message_content, MessageTextContent):
            print(f"{data_point.role}: {last_message_content.text.value}")

    # [END list_messages]
    print(f"Messages: {messages}")


Created agent, agent ID: asst_BGSKtvtTxSBS1thtLbJGnEIE
Created thread, thread ID: thread_V23r3H33qEPtVQljNeQsriet
Created message, message ID: msg_8aDcsIDGULkOWBm2H7xb2cWq
Run status: completed
Deleted agent
user: Hello, tell me a joke
assistant: Sure! Here's a light-hearted joke for you:

Why don't scientists trust atoms?

Because they make up everything!
Messages: {'object': 'list', 'data': [{'id': 'msg_Egmqk9EofpfyvFzsDmPW5iGg', 'object': 'thread.message', 'created_at': 1735236144, 'assistant_id': 'asst_BGSKtvtTxSBS1thtLbJGnEIE', 'thread_id': 'thread_V23r3H33qEPtVQljNeQsriet', 'run_id': 'run_99FCkisMjAGtekq78mrF6Nze', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "Sure! Here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_8aDcsIDGULkOWBm2H7xb2cWq', 'object': 'thread.message', 'created_at': 1735236143, 'assistant_id': None, 'thread_id'